# Configurações Iniciais

Os arquivos utilizados na célula abaixo estão localizados em um drive privado, eles só podem ser baixados através do link. Ao rodar a célula, esse processo será realizado de maneira automática e imediata.

In [ ]:
# @title Execute essa célula para baixar os arquivos e bibliotecas necessárias
!pip install --upgrade --no-cache-dir gdown

#Preditor
!gdown 1xUota1_BuTQh0xKJxVbmvpkvesQAAvW5

#Ordinal Encoder
!gdown 1JwFoGztRziZd7_aTlpm1cK5MqXGMqmuX

#Modelo-XGB
!gdown 1-RqWajeudwq8wJV5aS02LHRWHk4kNel3

#Modelo-RF
!gdown 10Atdv1dDtplPELqjYCDCKY6ENR0jJjFY

#Modelo-GBR
!gdown 1jy1F29zzwT_dGBT8Hdpx7iFu5vWg1zjQ

#Meta Modelo
!gdown 1Eo6Fi1gUzq6m1MtKUzh1z056XYEEYX9N

#Tabela_sku_dict
!gdown 1yqzxmzl5QUZGOJaLVsCAFYd8T12csyGg

# Importando as bibliotecas necessárias
!pip install numpy=='1.24.3'
!pip install --upgrade pandas
!pip install --upgrade xgboost
!pip install joblib=='1.2.0'
!pip install --upgrade scikit-learn

from joblib import load
from datetime import datetime, timedelta
from math import *
import skuPreditor as pr
import pandas as pd
import numpy as np
import warnings

#Carregando modelos e tabela
modelo = load("meta-model.joblib")

modelo_rf = load("modelo-rf.joblib")

modelo_gbr = load("modelo-gbr.joblib")

modelo_xgb = load("modelo-xgb.joblib")

encoder_sku = load('ordinal-encoder.joblib')

tabela_sku_dict = pd.read_csv("tabela_sku_dict.csv", sep =",")

warnings.filterwarnings('ignore')

# Input
Nessa seção é realizada a exportação dos modelos, carregamento e a possibilidade de fornecer dados de entrada para as predições.

Devido ao tamanho dos arquivos, é necessário rodar uma única vez os seguintes blocos de códigos do notebook Entrega Acadêmica para obtê-los:
`Configurações Iniciais, Pré Processamento, Ensemble e Dump, situados no notebook`.


Coloque eles na pasta já criada "modelos".

In [8]:
#@title Execute a célula de código abaixo para *realizar* a previsão.

# Criando um dicionário com que relaciona os SKUs e suas informações fixas
sku_dict = pr.Sku.to_dict(tabela_sku_dict)

# Definindo inputs
input_data=input('Qual é o período, em dias, que deseja realizar a previsão? \n')
data_inicial = input('A previsão deve começar em que data? Utilize o formato: ano-mês-dia, como em: 2023-07-04 \n')
print('\n')

#Função para fazer as previsões com base no input
def next_days(sku, start_date_str):
    # Convertendo data
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')

    # Lista para armazenar as próximas datas
    sales_prediction = {}

    # Calcula as próximas datas dentro do range
    for i in range(0, int(input_data)):
        next_date = start_date + timedelta(days=i)

        # Converte o objeto datetime para uma string no formato 'yyyy-MM-dd'
        next_date_str = next_date.strftime('%Y-%m-%d')
        new_instance = pr.SalesPredictor.create_new_instance(
            sku=sku,
            sku_dict=sku_dict,
            a_date=next_date_str
        )

        #Cria uma nova instância
        new_instance = new_instance.loc[:, ['unit_price', 'sku_length', 'sku_weight',
       'avg_website_visits_last_week', 'supplier_delivery_time',
       'color_encoded', 'sku_encoded', 'has_stock_encoded', 'ano', 'dia',
       'dia_da_semana', 'semana_do_ano', 'dia_do_ano']]
        new_instance

        # Faz as predições com os 3 modelos na nova instâcia
        pred1 = modelo_rf.predict(new_instance)
        pred2 = modelo_gbr.predict(new_instance)
        pred3 = modelo_xgb.predict(new_instance)

        #Juntando as predições
        X_instance = np.column_stack((pred3, pred1, pred2))

        #Gerando a predição com o modelo meta nas melhores colunas dos modelos previstos
        prediction = modelo.predict(X_instance)

        sales_prediction[next_date_str] = round(prediction[0])

    return sales_prediction

#####Previsão cumulativa:

soma_total = 0
data = []

for valor in sku_dict:
    # Revertendo o Ordinal Encoder
    valor_2d = np.array(valor).reshape(-1, 1)
    original_sku = encoder_sku.inverse_transform(valor_2d)

    prever = next_days(sku=valor, start_date_str= data_inicial)

    soma_dos_valores = sum(prever.values())

    # Armazena os dados em uma lista
    data.append([original_sku, soma_dos_valores])

    soma_total += soma_dos_valores

# Cria um DataFrame pandas com os dados
print(f'Total de cada sku vendido durante {input_data} dia(s), incluindo a data inicial {data_inicial}:')

df_summed = pd.DataFrame(data, columns=['SKU Original', 'Itens vendidos nos dias informados'])

display(df_summed)
print('\n')

##### Previsão por dia

# Cria um dicionário para armazenar as previsões para cada dia
predictions_per_day = {}

for valor in sku_dict:
    # Revertendo o Ordinal Encoder
    valor_2d = np.array(valor).reshape(-1, 1)
    original_sku = encoder_sku.inverse_transform(valor_2d)

    # Fazendo a previsão
    prever = next_days(sku=valor, start_date_str=data_inicial)

    # Adiciona as previsões ao dicionário
    for date, prediction in prever.items():
        if date not in predictions_per_day:
            predictions_per_day[date] = []
        predictions_per_day[date].append((original_sku, prediction))

# Cria um DataFrame vazio
data = []

# Preenchendo o DataFrame com os dados das previsões
for date, predictions in sorted(predictions_per_day.items()):
    for original_sku, prediction in predictions:
        data.append([date, original_sku, prediction])

# Criando um DataFrame com os dados
df_predictions = pd.DataFrame(data, columns=["Data", "SKU", "Items Vendidos"])

print(f'Número por dia de vendas de cada sku durante {input_data} dia(s), incluindo a data inicial {data_inicial}:')
display(df_predictions)


Qual é o período, em dias, que deseja realizar a previsão? 
1
A previsão deve começar em que data? Utilize o formato: ano-mês-dia, como em: 2023-07-04 
2023-09-04


Total de cada sku vendido durante 1 dia(s), incluindo a data inicial 2023-09-04:


,SKU Original,Itens vendidos nos dias informados
0,[[GU364MA69CWSMOB-1076165]],0
1,[[MO173CH44RCDMOB-998807]],0
2,[[LE761CH61UOWMOB-983937]],0
3,[[MA566UP68MFJMOB-943243]],0
4,[[ES745UP18PUZMOB-790516]],0
...,...,...
830,[[MO173UP15VAUMOB-742584]],12
831,[[RU994CL95WDWMOB-555452]],34
832,[[LI582UP22JZHMOB-204089]],11
833,[[DE350UP94AFPMOB-592995]],62


Número por dia de vendas de cada sku durante 1 dia(s), incluindo a data inicial 2023-09-04:


,Data,SKU,Items Vendidos
0,2023-09-04,[[GU364MA69CWSMOB-1076165]],0
1,2023-09-04,[[MO173CH44RCDMOB-998807]],0
2,2023-09-04,[[LE761CH61UOWMOB-983937]],0
3,2023-09-04,[[MA566UP68MFJMOB-943243]],0
4,2023-09-04,[[ES745UP18PUZMOB-790516]],0
...,...,...,...
830,2023-09-04,[[MO173UP15VAUMOB-742584]],12
831,2023-09-04,[[RU994CL95WDWMOB-555452]],34
832,2023-09-04,[[LI582UP22JZHMOB-204089]],11
833,2023-09-04,[[DE350UP94AFPMOB-592995]],62
